In [4]:
import time
from sklearn import datasets
import numpy as np
import pandas as pd

import cml.metrics_v1 as metrics

from predict_with_metrics import predict

# Number of examples
N_SAMPLES = 100

# Load Iris dataset
iris = datasets.load_iris()
feature_min = iris.data.min(axis=0)
feature_max = iris.data.max(axis=0)

# Generate a random synthetic sample
synthetic_sample = np.random.uniform(feature_min, feature_max, size=(N_SAMPLES, iris.data.shape[1])) # (100, 4)
synthetic_labels = np.random.choice(iris.target, size=N_SAMPLES) # (100, )

# Record the current time so we can retrieve the metrics
# tracked for these calls.
start_timestamp_ms=int(round(time.time() * 1000))

# Simulate batch inference
uuids = []

# Iterate over each row in the synthetic_sample array
for sample in synthetic_sample:
    single_row_df = pd.DataFrame({"inputs": [sample.tolist()]})
    output = predict(single_row_df)
    # Record the UUID of each prediction for correlation with ground truth.
    uuids.append(output["uuid"])

In [5]:
# Record the current time.
end_timestamp_ms=int(round(time.time() * 1000))

# We can now use the read_metrics function to read the metrics we just
# generated into the current session, by querying by time window.
data = metrics.read_metrics(start_timestamp_ms=start_timestamp_ms,
                            end_timestamp_ms=end_timestamp_ms, dev=True)["metrics"]

# Now, ground truth is known and we want to track the true value
# corresponding to each prediction above.
synthetic_labels = np.random.choice(iris.target, size=N_SAMPLES)

# Track the true values alongside the corresponding predictions
# with track_delayed_metrics function
n = len(synthetic_labels)
for i in range(n):
    ground_truth = synthetic_labels[i]
    metrics.track_delayed_metrics({"actual_result": str(ground_truth)}, uuids[i], dev=True)

In [8]:
data[:10]

[{'modelDeploymentCrn': 'crn:cdp:ml:::workspace:dev/model-deployment',
  'modelBuildCrn': 'crn:cdp:ml:::workspace:dev/model-build',
  'modelCrn': 'crn:cdp:ml:::workspace:dev/model-deployment',
  'startTimeStampMs': 1739926103995,
  'endTimeStampMs': 1739926103996,
  'predictionUuid': '3f5bcd30-e017-45ac-b20e-02f01ca5c8e3',
  'metrics': {'input': [4.6, 3.6, 1.0, 0.2], 'output': '0'}},
 {'modelDeploymentCrn': 'crn:cdp:ml:::workspace:dev/model-deployment',
  'modelBuildCrn': 'crn:cdp:ml:::workspace:dev/model-build',
  'modelCrn': 'crn:cdp:ml:::workspace:dev/model-deployment',
  'startTimeStampMs': 1739926219305,
  'endTimeStampMs': 1739926219305,
  'predictionUuid': '85657c7e-44c2-4cba-948a-dd753472cff2',
  'metrics': {'actual_result': '0',
   'input': [5.402312569399343,
    3.8727971928374045,
    4.610248175740823,
    1.1782076628411633],
   'output': '2'}},
 {'modelDeploymentCrn': 'crn:cdp:ml:::workspace:dev/model-deployment',
  'modelBuildCrn': 'crn:cdp:ml:::workspace:dev/model-buil